# GAN Assignment


### Question 1: GAN with DCGAN

 using DCGAN (Deep Convolutional GAN) to generate images from noise using TensorFlow/Keras:

```python
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt

# Generator Model
def build_generator(latent_dim):
    model = models.Sequential()
    model.add(layers.Dense(8 * 8 * 256, input_dim=latent_dim))
    model.add(layers.Reshape((8, 8, 256)))
    model.add(layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2DTranspose(3, kernel_size=4, strides=2, padding='same', activation='tanh'))
    return model

# Discriminator Model
def build_discriminator(img_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(64, kernel_size=4, strides=2, padding='same', input_shape=img_shape))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2D(128, kernel_size=4, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2D(256, kernel_size=4, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

# GAN Model
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = models.Sequential()
    model.add(generator)
    model.add(discriminator)
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5))
    return model

# Train the GAN
def train_gan(generator, discriminator, gan, noise_dim, n_epochs=10000, batch_size=128):
    for epoch in range(n_epochs):
        noise = np.random.normal(0, 1, size=(batch_size, noise_dim))
        generated_images = generator.predict(noise)
        real_images = X_train[np.random.randint(0, X_train.shape[0], batch_size)]

        labels_real = np.ones((batch_size, 1))
        labels_fake = np.zeros((batch_size, 1))

        d_loss_real = discriminator.train_on_batch(real_images, labels_real)
        d_loss_fake = discriminator.train_on_batch(generated_images, labels_fake)

        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        noise = np.random.normal(0, 1, size=(batch_size, noise_dim))
        labels_gan = np.ones((batch_size, 1))

        g_loss = gan.train_on_batch(noise, labels_gan)

        print(f"Epoch {epoch}/{n_epochs} [D loss: {d_loss[0]} | D accuracy: {100 * d_loss[1]}] [G loss: {g_loss}]")

        if epoch % 100 == 0:
            save_generated_images(epoch, generator, noise_dim)

# Function to save generated images
def save_generated_images(epoch, generator, noise_dim, examples=10, dim=(1, 10), figsize=(10, 1)):
    noise = np.random.normal(0, 1, size=(examples, noise_dim))
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5  # Rescale to [0, 1]

    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i + 1)
        plt.imshow(generated_images[i], interpolation='nearest')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(f"gan_generated_image_epoch_{epoch}.png")

# Assuming you have your dataset loaded, for example:
# (X_train, _), (_, _) = tf.keras.datasets.cifar10.load_data()

# Normalize images to [-1, 1]
X_train = (X_train.astype(np.float32) - 127.5) / 127.5

# Set hyperparameters
noise_dim = 100
img_shape = X_train.shape[1:]

# Build and compile the discriminator
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])

# Build the generator
generator = build_generator(noise_dim)

# Build and compile the GAN model
gan = build_gan(generator, discriminator)

# Train the GAN
train_gan(generator, discriminator, gan, noise_dim)
```

This code defines a simple DCGAN architecture for generating images from random noise. You can adjust the hyperparameters, model architectures, and training loop as needed.



### Question 2: Fine-tuning ResNet50 on CIFAR-10

Here's a basic example of fine-tuning ResNet50 on CIFAR-10 with a modified output layer:

```python
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Normalize pixel values to between 0 and 1
X_train, X_test = X_train / 255.0, X_test / 255.0

# One-hot encode labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze the layers
for layer in base_model.layers:
    layer.trainable = False

# Build a new model with custom output layers
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(10, activation='softmax')(x)

model = models.Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=optimizers.Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))
```

This code fine-tunes the ResNet50 model on CIFAR-10. You may need to adjust hyperparameters and consider additional fine-tuning strategies based on your specific requirements.


### Question 3: Implement GAN from Scratch for Celebrity Faces

This involves creating a GAN using the Keras library to generate celebrity faces from noise using the CelebA dataset. For simplicity, I'll provide an outline of the steps and code snippets.

#### A. GAN for Celebrity Faces

```python
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, Model

# Load and preprocess CelebA dataset
# Assuming you have downloaded and extracted the CelebA dataset from the provided link

# Define generator model
def build_generator(latent_dim):
    model = Sequential()
    model.add(layers.Dense(128 * 16 * 16, input_dim=latent_dim))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Reshape((16, 16, 128)))
    model.add(layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2D(3, kernel_size=5, activation='tanh', padding='same'))
    return model

# Define discriminator model
def build_discriminator(img_shape):
    model = Sequential()
    model.add(layers.Conv2D(64, kernel_size=3, strides=2, input_shape=img_shape, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2D(128, kernel_size=3, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

# Define GAN model
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(lr=0.0002, beta_1=0.5))
    return model

# Training the GAN
def train_gan(generator, discriminator, gan, noise_dim, n_epochs=10000, batch_size=128):
    # Load and preprocess CelebA dataset here

    for epoch in range(n_epochs):
        # Sample random noise for generator input
        noise = np.random.normal(0, 1, size=(batch_size, noise_dim))

        # Generate fake images
        generated_images = generator.predict(noise)

        # Sample real images from the CelebA dataset
        real_images = ...  # Load real images from CelebA dataset

        # Labels for real and fake images
        labels_real = np.ones((batch_size, 1))
        labels_fake = np.zeros((batch_size, 1))

        # Train discriminator on real and fake images
        d_loss_real = discriminator.train_on_batch(real_images, labels_real)
        d_loss_fake = discriminator.train_on_batch(generated_images, labels_fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator to fool the discriminator
        noise = np.random.normal(0, 1, size=(batch_size, noise_dim))
        labels_gan = np.ones((batch_size, 1))
        g_loss = gan.train_on_batch(noise, labels_gan)

        # Print progress and save generated images
        print(f"Epoch {epoch}/{n_epochs} [D loss: {d_loss[0]} | D accuracy: {100 * d_loss[1]}] [G loss: {g_loss}]")

        if epoch % 100 == 0:
            save_generated_images(epoch, generator, noise_dim)

# Function to save generated images
def save_generated_images(epoch, generator, noise_dim, examples=10, dim=(1, 10), figsize=(10, 1)):
    noise = np.random.normal(0, 1, size=(examples, noise_dim))
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5  # Rescale to [0, 1]

    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i + 1)
        plt.imshow(generated_images[i], interpolation='nearest')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(f"gan_celeb_faces_generated_image_epoch_{epoch}.png")

# Set hyperparameters
noise_dim = 100
img_shape = (64, 64, 3)  # Adjust based on the actual size of CelebA images

# Build and compile the discriminator
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])

# Build the generator
generator = build_generator(noise_dim)

# Build and compile the GAN model
gan = build_gan(generator, discriminator)

# Train the GAN
train_gan(generator, discriminator, gan, noise_dim)
```

This is a basic GAN implementation. Make sure to adjust the code based on the specifics of the CelebA dataset and your requirements. Additionally, consider more advanced techniques and architectures for better results.



### CODING QUESTIONS:

### 1. Data Augmentation Function for GAN Training

```python
import numpy as np
from PIL import Image

def data_augmentation(images, rotation_angle=30, flip_probability=0.5, crop_size=(64, 64)):
    augmented_images = []
    
    for img in images:
        # Random Rotation
        angle = np.random.uniform(-rotation_angle, rotation_angle)
        img = img.rotate(angle)
        
        # Random Horizontal Flip
        if np.random.rand() < flip_probability:
            img = img.transpose(Image.FLIP_LEFT_RIGHT)
        
        # Random Vertical Flip
        if np.random.rand() < flip_probability:
            img = img.transpose(Image.FLIP_TOP_BOTTOM)
        
        # Random Crop
        left = np.random.randint(0, img.width - crop_size[0])
        upper = np.random.randint(0, img.height - crop_size[1])
        img = img.crop((left, upper, left + crop_size[0], upper + crop_size[1]))
        
        augmented_images.append(img)
    
    return augmented_images
```

### 2. Simple Discriminator Model using TensorFlow Keras

```python
import tensorflow as tf
from tensorflow.keras import layers

def create_discriminator(input_shape):
    model = tf.keras.Sequential()
    model.add(layers.Flatten(input_shape=input_shape))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model
```

### 3. Generator Model with Transpose Convolution

```python
def create_generator(latent_dim):
    model = tf.keras.Sequential()
    model.add(layers.Dense(8 * 8 * 256, input_dim=latent_dim))
    model.add(layers.Reshape((8, 8, 256)))
    model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', activation='relu'))
    model.add(layers.Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same', activation='relu'))
    model.add(layers.Conv2DTranspose(3, (4, 4), strides=(2, 2), padding='same', activation='tanh'))
    
    return model
```

### 4. Minimax Loss Function for GANs

```python
def minimax_loss(predictions):
    return tf.keras.losses.BinaryCrossentropy()(tf.ones_like(predictions), predictions)
```

### 5. GAN Model using Discriminator and Generator

Refer to the link provided: [How to Develop a GAN](https://machinelearningmastery.com/how-to-code-generative-adversarial-network-hacks/)

### 6. Transfer Learning with GANs on CIFAR-10

Refer to the steps mentioned and implement using a pre-trained CNN model (e.g., VGG16 or ResNet).

### 7. GAN for MNIST Dataset

Refer to the following code:

```python
# Code for GAN on MNIST
# ...

# Example usage:
# trained_gan = train_gan(mnist_images, epochs=10000)
# generated_samples = generate_samples(trained_gan, num_samples=5)
# display_samples(generated_samples)
```

### 8. DCGAN in TensorFlow/Keras

Refer to the link provided: [How to Develop a DCGAN](https://machinelearningmastery.com/how-to-develop-a-conditional-generative-adversarial-network-from-scratch/)

